# PCA of EEG Features (Dortmund Dataset)
This notebook describes the process of applying PCA to EEG-derived features, exploring dimensionality reduction, and preparing data for clustering.

- Objective: Reduce high-dimensional EEG features to a lower-dimensional space to explore participant variability.
- Dataset: Dortmund dataset (608 participants, multiple EEG features).
- Goals:
  1. Understand structure of the features.
  2. Perform PCA for dimensionality reduction.
  3. Explore variance explained by principal components.
  4. Prepare data for clustering and visualization.

## Load Data
Load the Dortmund dataset and inspect the first few rows.

In [ ]:
import pandas as pd
import numpy as np

dort = pd.read_csv("./dataset/Dortmund_features.csv", index_col=0)  # Keep subject IDs as index!
age_sex_dort = pd.read_csv("./dataset/Dortmund_age&sex.csv", index_col=0)
# Extract age and sex
age_dort = age_sex_dort['age'].values
sex_dort = age_sex_dort['sex'].values

lemon = pd.read_csv("./dataset/Lemon_features.csv", index_col=0)  # Keep subject IDs as index!
age_sex_lemon = pd.read_csv("./dataset/Lemon_age&sex.csv", index_col=0)
# Extract age and sex
age_lemon = age_sex_lemon['age_group'].values # note ages are ranges in Lemon
sex_lemon = age_sex_lemon['sex'].values

## 1. Exploratory Data Analysis
- Preview first few rows and numeric columns.
- Check for missing values.
- Examine feature distributions.

In [ ]:
# Print dataset shapes
print(f"Dortmund_features.csv shape: {dort.shape}")
print(f"LEMON_features.csv shape: {lemon.shape}\n")

dort_cols = set(dort.columns)
lemon_cols = set(lemon.columns)

print("Features only in Dortmund:")
print(dort_cols - lemon_cols, "\n")

print("Features only in LEMON:")
print(lemon_cols - dort_cols, "\n")


All features are shared in Dortmund and Lemon

1. Scatter Plot of First Two Raw Features


In [ ]:
import matplotlib.pyplot as plt

# Select only numeric columns (exclude IDs or non-numeric)
numeric_data = dort.select_dtypes(include=['float64', 'int64'])
dort_numeric = numeric_data.values

# Define features (all numeric columns)
features = dort_numeric  # shape: (participants, features)

# Print what we are plotting
print("Plotting the first two numeric features:")
print(f"Feature 1: {numeric_data.columns[0]}")
print(f"Feature 2: {numeric_data.columns[1]}\n")

# Scatter plot of first two features
plt.figure(figsize=(8,6))
plt.scatter(features[:, 0], features[:, 1], alpha=0.7)
plt.xlabel(numeric_data.columns[0])
plt.ylabel(numeric_data.columns[1])
plt.title("Scatter plot of first two features")
plt.grid(True)
plt.show()


2. Feature-Age/Sex Relationship
-   **Age** — plotted using scatter plots  
-   **Sex** — plotted using boxplots (female vs male)

In [ ]:
# Numeric Dortmund features
dort_numeric = dort.select_dtypes(include=['float64', 'int64'])
num_features_to_plot = 6

# -------------------------
# Plot features vs Age
# -------------------------
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 8))
axes = axes.flatten()

for i in range(num_features_to_plot):
    axes[i].scatter(age_dort, dort_numeric.iloc[:, i], alpha=0.7)
    axes[i].set_title(dort_numeric.columns[i])
    axes[i].set_xlabel("Age")
    axes[i].set_ylabel("Feature Value")

plt.suptitle("Dortmund Features vs Age", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# -------------------------
# Plot features vs Sex
# -------------------------
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 8))
axes = axes.flatten()

for i in range(num_features_to_plot):
    axes[i].boxplot([dort_numeric.iloc[sex_dort=='F', i], dort_numeric.iloc[sex_dort=='M', i]],
                    labels=['F','M'])
    axes[i].set_title(dort_numeric.columns[i])
    axes[i].set_ylabel("Feature Value")

plt.suptitle("Dortmund Features vs Sex", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


## 2. Principal Component Analysis (PCA)
- Reduce dimensionality to an optimal number of PCs for clustering while optionally visualizing in 2D.
- Examine explained variance, singular values, and component loadings to understand feature contributions.


1. Standardize Features
- PCA is sensitive to scale, so all numeric features are standardized to mean = 0 and variance = 1 before PCA.

2. Optimal PCA Computation
- Compute full PCA and determine the minimum number of PCs needed to reach a threshold variance (default 80%).
- Plot cumulative explained variance and scree plot to visualize the variance captured by each component.

3.  PCA Component Loadings & Transformed Data
- Loadings indicate how strongly each original feature contributes to each principal component.
- Transformed PCA data (scores) are used for clustering.
- Optionally export loadings and PCA-transformed dataset for further analysis.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

def run_optimal_pca(data, threshold=0.8, dataset_name="Dataset", export=False):
    """
    Standardizes data, computes full PCA, finds optimal PCs using cumulative variance,
    plots PCA results, and returns PCA-transformed data and components.
    
    Parameters:
        data : pandas DataFrame (numeric)
        threshold : float, e.g. 0.8 for 80% variance
        dataset_name : str (used in plot titles)
        
    Returns:
        pca_optimal_components : np.array (PCA-transformed data)
        pca_model_optimal : PCA object fitted with optimal n_components
        num_pcs_threshold : int (optimal number of PCs)
        explained_variance_full : np.array (all PCs variance)
        components_optimal : np.array (loadings)
    """

    # 1. Standardize
    scaled = StandardScaler().fit_transform(data)

    # 2. Full PCA
    pca_full = PCA()
    pca_full.fit(scaled)

    explained_variance_full = pca_full.explained_variance_ratio_
    cumulative = np.cumsum(explained_variance_full)

    # 3. Determine optimal PCs
    num_pcs_threshold = np.argmax(cumulative >= threshold) + 1

    # --- Plotting ---
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Cumulative plot
    axes[0].plot(range(1, len(cumulative)+1), cumulative*100, marker='o')
    axes[0].axvline(num_pcs_threshold, color='r', linestyle='--',
                    label=f'{int(threshold*100)}% at PC{num_pcs_threshold}')
    axes[0].set_title(f"{dataset_name}: Cumulative Explained Variance")
    axes[0].set_xlabel("Number of Principal Components")
    axes[0].set_ylabel("Cumulative Explained Variance (%)")
    axes[0].legend()
    axes[0].grid(True)

    # Scree plot
    axes[1].plot(range(1, len(explained_variance_full)+1),
                 explained_variance_full*100, marker='o', alpha=0.7)
    axes[1].axhline(explained_variance_full[num_pcs_threshold-1]*100,
                    color='r', linestyle='--',
                    label=f'PC{num_pcs_threshold} variance')
    axes[1].set_title(f"{dataset_name}: Scree Plot")
    axes[1].set_xlabel("Principal Component")
    axes[1].set_ylabel("Explained Variance (%)")
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()

    # 4. Run optimal PCA
    pca_optimal = PCA(n_components=num_pcs_threshold)
    transformed = pca_optimal.fit_transform(scaled)

    # --- Printing ---
    print(f"\n=== {dataset_name}: Optimal PCA Summary ===")
    print(f"Optimal number of PCs for {int(threshold*100)}% variance: {num_pcs_threshold}")
    print(f"Variance explained by PC1: {explained_variance_full[0]*100:.2f}%")
    print(f"Total variance explained by {num_pcs_threshold} PCs: {cumulative[num_pcs_threshold-1]*100:.2f}%\n")

    print("\n=== Explained Variance of Optimal PCs ===")
    for i, v in enumerate(pca_optimal.explained_variance_ratio_, start=1):
        print(f"PC{i}: {v*100:.2f}%")

    print("\n=== First 5 Singular Values ===")
    print(pca_optimal.singular_values_[:5])

    print("\n=== PCA Loadings (First 3 Features Per PC) ===")
    for i, pc in enumerate(pca_optimal.components_, start=1):
        print(f"PC{i}: {pc[:3]}")


    # --------------------------- #
    # 5. Optional export to CSV
    # --------------------------- #
    # --------------------------- #
    # 5. Optional export to CSV
    # --------------------------- #
    if export:
        # --- PCA Components (Loadings) ---
        # Rows = PCs, Columns = original features
        # This shows HOW MUCH each original feature contributes to each PC
        loadings_df = pd.DataFrame(
            pca_optimal.components_,
            columns=data.columns,
            index=[f"PC{i+1}" for i in range(num_pcs_threshold)]
        )
        loadings_path = f"dataset/{dataset_name}_pca_loadings.csv"
        loadings_df.to_csv(loadings_path, index=True)

        # --- PCA Transformed Data ---
        # Rows = Subjects, Columns = PCs
        # This is what you use for CLUSTERING
        transformed_df = pd.DataFrame(
            transformed,
            index=data.index, # Keep original subject IDs!
            columns=[f"PC{i+1}" for i in range(num_pcs_threshold)]
        )
        transformed_path = f"dataset/{dataset_name}_pca.csv"
        transformed_df.to_csv(transformed_path, index=True)

        print(f"\n✓ Exported loadings to:     {loadings_path}")
        print(f"✓ Exported PCA scores to:   {transformed_path}")
        print(f"  Subject IDs preserved: {transformed_df.index[:3].tolist()}... (first 3)")

    return transformed, pca_optimal, num_pcs_threshold, explained_variance_full, pca_optimal.components_


OPTIMAL PCA (DORTMUND DATASET):

data_pca      → The PCA-transformed dataset (participants × optimal PCs).
                These are the coordinates of each subject in PCA space.

data_model    → The fitted PCA model object.
                Contains components_, explained_variance_ratio_, singular_values_, etc.

data_nPC      → The optimal number of principal components selected automatically.
                Determined by reaching the cumulative variance threshold (e.g., 80%).

data_var      → Explained variance ratio of each selected PC.
                Tells you how much variance each principal component captures.

data_loadings → PCA loadings (components): contribution of each original feature to each PC.
                Shape = (nPC × n_features). Used for interpretation and clustering.

In [ ]:
dort_numeric = dort.select_dtypes(include=['float64', 'int64'])
dort_pca, dort_model, dort_nPC, dort_var, dort_loadings = run_optimal_pca(
    dort_numeric,
    threshold=0.80,
    dataset_name="Dortmund",
    export=True
)

OPTIMAL PCA (LEMON DATASET):

In [ ]:
lemon_numeric = lemon.select_dtypes(include=['float64', 'int64'])
lemon_pca, lemon_model, lemon_nPC, lemon_var, lemon_loadings = run_optimal_pca(
    lemon_numeric,
    threshold=0.8,
    dataset_name="LEMON",
    export=True
)


## 3. PCA Stability Analysis: Bootstrap 
To assess the stability of PCA-derived feature subspaces for two datasets (Dortmund and Lemon) - understand how robust the principal components are to sampling variability

1. **Bootstrap Resampling:**  
 - For each dataset, we create `n_boot` resampled datasets by sampling subjects with replacement.
- PCA is applied to each resampled dataset, keeping components that explain 99% of the variance.

2. **Subspace Comparison:**  
- For each pair of bootstrap PCA results within the same dataset, we compute the singular values of the matrix product between their component matrices.
- Singular values close to 1 = nearly identical subspaces (high stability)
- Also compute singular values between Dortmund and Lemon bootstraps to see how similar the feature subspaces are across datasets.

In [ ]:
from numpy.linalg import svd
from sklearn.utils import resample

def bootstrap_pca(X, n_boot, variance_threshold=0.99):
    """
    Compute PCA on multiple bootstrap resamples of the dataset to assess stability.
    
    Parameters:
    - X: array-like, shape (n_samples, n_features)
        The data matrix to analyze.
    - n_boot: int
        Number of bootstrap resamples.
    - variance_threshold: float
        Fraction of variance to retain in PCA.
    
    Returns:
    - components_boot: list of arrays
        Each array contains PCA components for one bootstrap sample (shape: n_features x n_components).
    - min_n_comp: int
        Minimum number of components across all bootstrap samples.
    """

    n_samples, n_features = X.shape
    components_boot = []
    min_n_comp = n_features

    for i in range(n_boot):
        # Resample rows (participants) with replacement
        X_resampled = resample(X, random_state=i)

        # Standardize resampled data
        X_scaled = StandardScaler().fit_transform(X_resampled)

        # Fit PCA to bootstrap-sampled data
        pca = PCA(n_components=variance_threshold)
        pca.fit(X_scaled)

        # Store the component vectors
        components_boot.append(pca.components_.T)   # shape: features × components
        
        # Keep track of minimum number of components across bootstraps
        min_n_comp = np.min((min_n_comp, len(pca.components_)))

    return components_boot, min_n_comp



def singular_values(mat1, mat2):
    """
    Measure similarity between two PCA component matrices using singular values.
    
    Parameters:
    - mat1, mat2: arrays of shape (n_features, n_components)
    
    Returns:
    - Array of singular values representing alignment of subspaces.
    """

    M = mat1.T @ mat2
    _, s, _ = svd(M)
    return s

In [ ]:
# Number of bootstrap iterations
n_boot = 50  # For readability in examples; typically use >=100 in actual analysis

# Run bootstrap PCA on Dortmund and LEMON datasets
dort_comp, min_dort = bootstrap_pca(dort, n_boot)
lem_comp, min_lem = bootstrap_pca(lemon, n_boot)

# Find the minimum number of components across all bootstraps/datasets
min_n_comp = np.min((min_dort, min_lem))

# Trim each bootstrap PCA result to have the same number of components to allow to compare subspaces using singular values
dort_comp = [boot[:, :min_n_comp] for boot in dort_comp]
lem_comp = [boot[:, :min_n_comp] for boot in lem_comp]
# have aligned PCA bases of shape (features, min_n_com

In [ ]:
# Initialize dictionary to store singular values
results = {
    "within_Dortmund": {},
    "within_Lemon": {},
    "between_DL": {}
}

# Compute singular values within bootstraps:
# within Dortmund
for i, boot1 in enumerate(dort_comp):
    for j, boot2 in enumerate(dort_comp):
        if j > i: # only upper-triangle comparisons to avoid duplicates
            results["within_Dortmund"][f"boots {i}-{j}"] = singular_values(boot1, boot2)

# within Lemon
for i, boot1 in enumerate(lem_comp):
    for j, boot2 in enumerate(lem_comp):
        if j > i:
            results["within_Lemon"][f"boots {i}-{j}"] = singular_values(boot1, boot2)

# cross dataset
for i, boot1 in enumerate(dort_comp):
    for j, boot2 in enumerate(lem_comp):
        if j >= i: # compare each Dortmund bootstrap to corresponding or later LEMON bootstraps
            results["between_DL"][f"boots {i}-{j}"] = singular_values(boot1, boot2)


### Results

##### Within-Dataset Stability (Lemon-Lemon or Dortmund): 
All within-Lemon bootstrap similarity distributions fall extremely close to **1.0**, with most values in the range **0.98–1.00**.  
This indicates:
- *High cluster stability:* each component is internally consistent.
- *Robust centroid estimation:* resampling does not meaningfully shift the cluster centers.
- *Well-separated structure:* there is no sign of component collapse or merging.

In short, the Lemon dataset exhibits **very strong internal reproducibility**.

In [ ]:
# Display results
# - High singular values (~1) indicate strong alignment of PCA subspaces
# - Lower singular values indicate differences between subspaces

results["within_Dortmund"]

In [ ]:
results["within_Lemon"]

##### Cross-Dataset Similarity (Dortmund vs. Lemon)
Cross-dataset bootstrap similarities fall in the range:
- **~0.20–0.27** at the high end  
- rapidly dropping toward **0.01–0.0001**

This pattern indicates:

1. **Minimal structural alignment between the datasets.**
   The components extracted from Dortmund do not resemble those from Lemon.

2. **No evidence of shared component geometry.**
   If the datasets had corresponding factors or clusters, we would expect cross-dataset similarities closer to **0.6–0.9**.  
   Instead, values are an order of magnitude lower.

3. **Clear separability.**
   The between-dataset similarity distributions never overlap with the within-dataset distributions (0.98–1.00 vs. 0.00–0.27).  
   This is strong evidence that the datasets express **distinct underlying patterns**.

In [ ]:
results["between_DL"]

### Visualising Bootstraping

In [ ]:
import seaborn as sns

def summarize_bootstrap_results_subplots(results_dicts, titles):
    """
    Plot multiple bootstrap singular value boxplots in subplots.

    results_dicts: list of results dictionaries (e.g., [within_Dortmund, within_Lemon, between_DL])
    titles: list of subplot titles corresponding to each dictionary
    """
    n_plots = len(results_dicts)
    fig, axes = plt.subplots(1, n_plots, figsize=(5*n_plots, 5), sharey=True)

    if n_plots == 1:
        axes = [axes]  # Ensure axes is iterable

    for ax, results_dict, title in zip(axes, results_dicts, titles):
        all_svals = np.array(list(results_dict.values()))
        sns.boxplot(data=all_svals, ax=ax)

        n_components = all_svals.shape[1]
        ax.set_xticks(range(n_components))
        ax.set_xticklabels([f"PC{i}" for i in range(1, n_components+1)])

        ax.set_xlabel("Principal Component")
        ax.set_title(title)
        ax.set_ylim(0,1.05)

        # Print mean singular values
        mean_svals = np.mean(all_svals, axis=0)
        print(f"{title} - Mean singular values per component:\n{mean_svals}\n")

    axes[0].set_ylabel("Singular Value")
    plt.tight_layout()
    plt.show()

# Call the function
summarize_bootstrap_results_subplots(
    [results["within_Dortmund"], results["within_Lemon"], results["between_DL"]],
    ["Within Dortmund PCA", "Within LEMON PCA", "Between Dortmund & LEMON PCA"]
)


In [ ]:
def lineplot_bootstrap_subplot(results_dicts, titles):
    """
    Plot multiple bootstrap singular value lineplots in subplots.

    results_dicts: list of results dictionaries (e.g., [within_Dortmund, within_Lemon, between_DL])
    titles: list of subplot titles corresponding to each dictionary
    """
    n_plots = len(results_dicts)
    fig, axes = plt.subplots(1, n_plots, figsize=(5*n_plots, 5), sharey=True)

    if n_plots == 1:
        axes = [axes]  # Ensure axes is iterable

    for ax, results_dict, title in zip(axes, results_dicts, titles):
        all_svals = np.array(list(results_dict.values()))
        for svals in all_svals:
            ax.plot(range(1, len(svals)+1), svals, marker='o', alpha=0.7)
        ax.set_xlabel("Principal Component")
        ax.set_title(title)
        ax.set_ylim(0,1.05)

    axes[0].set_ylabel("Singular Value")  # Only left-most plot needs y-axis label
    plt.tight_layout()
    plt.show()

# Call the function with your results
lineplot_bootstrap_subplot(
    [results["within_Dortmund"], results["within_Lemon"], results["between_DL"]],
    ["Within Dortmund PCA", "Within LEMON PCA", "Between Dortmund & LEMON PCA"]
)


In [ ]:
def summary_table_all(results_dicts, labels):
    
    for results_dict, label in zip(results_dicts, labels):
        all_svals = np.array(list(results_dict.values()))
        mean_svals = np.mean(all_svals, axis=0)
        std_svals = np.std(all_svals, axis=0)
        df = pd.DataFrame({
            "Component": range(1, len(mean_svals)+1),
            "Mean Singular Value": mean_svals,
            "Std Dev": std_svals
        })
        print(f"\nSummary for {label}:\n")
        display(df)

summary_table_all(
    [results["within_Dortmund"], results["within_Lemon"], results["between_DL"]],
    ["Within Dortmund", "Within LEMON", "Between Dortmund & LEMON"]
)


### PCA Stability and Cross-Dataset Comparison (Summary)

- **Within Dortmund:** Principal components are highly stable across bootstrap samples (singular values ≈ 0.98–0.999). PCA subspace is robust and reproducible.
- **Within LEMON:** Components are stable overall, though the least dominant components vary slightly more (singular values ≈ 0.64–0.99).
- **Between Dortmund and LEMON:** Singular values are very low (≈ 0.0001–0.25), indicating that PCA subspaces are dataset-specific and not shared.

### Conclusion:
- PCA is reliable within each dataset but differs between datasets.
- The same PCA cannot be used for both Dortmund and LEMON, so downstream analyses (e.g., clustering) should be performed on PCA fitted separately to each dataset.

### Next Steps (Optional)
If desired, the bootstrap distributions can be further summarized by:

- KL-divergence between distributions  
- p-values for separability  
- violin/ridge plots for publication  
- cluster dendrograms based on similarity